## Project Overview
### Identifying Enron persons of interest with machine learning

#### Project Goal

The goal of this project is to programatically determine correlations between the data obtained in the Enron case and whether a person was a 'person of interest' in the case.

If we could find a clear and consise way to determine whether someone is breaking the law in banking, we could use that methodology to automatically flag when a person should be investigated for possible active crimes.  Having personally worked in the financial industry, I know that there is a constant drive to catch a person who is violating rules or law, and to prevent the bank from losing money due to an employee's misconduct or malfeasance. 

This is data from the Enron case.  In it, we have information regarding how much each person was compensated, we calculate how often they emailed other persons in the data, and we know whether they were considered a person of interest in the case.  Out of the 35 persons of interest in the case, the data set contains data on 18 of them.  There are 143 persons in this data, 35 or 12.59% of which are persons of interest.  In other words, out of the 143 persons total, 35 are persons of interest and 125 are not persons of interest.

This specific data set is the result of discovery in an extensive legal case, so we can safely assume that any financial values not reported are zero and that all data that could be reported was reported.  While validating that the financials balanced vertically, I found two instances where the pdf with financial values was missing a dash for a zero value, which created the import process to set everything off a bit.  I created code to adjust those two records to properly reflect what's visually shown in the pdf.  I also noticed that the 'total' line imported, so I removed that from the data set as well.  While digging into other outliers in the data, I noticed both the name of a travel agency and an individual with only NaN or False data points.  I've also removed these in the interest of data quality.

#### Feature Selection
When choosing features, there's a tradeoff between bias and variance, or simplicity of fit versus goodness of fit.  Too few features chosen will create an oversimplified model, while too many features chosen will create an overfitted model.  The oversimplified model will pay too little attention to the training data, and the otherfitted model pays too much attention to the training data.  They both will not perform well on testing data, or data that is 'new' to the model.  The goal of feature selection is to find a balance between both, where the features that are used are those with the most impact on the data, and the model will have a low error rate on both data used in training and data used in testing.

In my initial testing, I created a feature to contain the sum of all compensation the individual received, as well as a feature to determine whether their total compensation was an outlier when compared to the other individuals in the data set.  These were interesting to graph, but did not show any visual promise of correlation between the feature and whether the person was a POI.

I eventually automatically selected my final features using MinMaxScaler and SelectKBest in a pipeline.  I tried various combinations of manually selecting features and using different appropriate feature selection methods, and this was the most effective of my trial selections. Because most of the features I was testing with SelectKBest were financial features, I tried both scaled and non-scaled testing.  Scaled testing performed slightly better overall, with slightly worse precision and slightly better recall.  Because I was trying to minimize the false negatives, I chose the scaled version as my final identifier.  I also used recall for the scoring method, as I wanted to associate a higher cost to false negatives than false positives.  If given the choice, I'd rather investigate someone who's innocent of fraud than miss someone who's guilty.  

The final features selected were salary, bonus, long_term_incentive, deferred_income, loan_advances, other, expenses, total_payments, exercised_stock_options, restricted_stock, total_stock_value, and from_poi_to_this_person. The lowest feature importance score in the set was 0.69 for exercised stock options, and the highest score in the set was 22.86 for salary.  All other features in the data set were dropped due to low feature importance scores.

#### Algorithm Selection
The algorithm I ended up using was an AdaBoostClassifier, using 50 estimators and a 0.9 learning rate.  I initially tried various manually selected features in an SVC, but did not have anything remotely resembling success.  I then tried using a Min Max Scaler, PCA, linear SVC and Gradient Boosting Classifier combined in a pipeline with f1 scoring in Grid Search CV.  This yielded better results, but did not prioritize minimizing false positives, so I then tried the same pipeline with recall scoring in Grid Search CV.  This was slightly better, but still under the goal of .3 minimum scores across the board.  I then tried using Select K Best and Ada Boost on unscaled features, with Grid Search CV utilizing recall scoring.  This again yielded better results, so I tried scaling, which provided values above the .3 mininum goal and performed better than the non-scaled classifier.

#### Algorithm Tuning
Tuning the parameters of the algorithm is a term to describe adjusting the model to achieve the best possible balance between performing well on the training data and performing well on testing data.  Again, this is where the bias-variance tradeoff comes into play.  The goal is to select the algorithm's parameters in such a way that we will have a model that, to paraphrase Albert Einstein, is as simple as possible, but no simpler.  An overfitted model will both have poor outcomes with a much higher error on the testing set than on the training set, as well as poor performance as each new feature added increases the computational cost of the model.

Rather than run a large number of combinations of parameters through trial and error, I used Grid Search CV to find the best estimator for each of the pipelines I created in my testing.  In order, using Grid Search CV for all, the worst to best performing pipelines were: 
  * Min Max Scaler, PCA, Linear SVC, and Gradient Boosting Classifier using f1 scoring
  * Min Max Scaler, PCA, Linear SVC, and Gradient Boosting Classifier using recall scoring
  * Select K Best and Ada Boost using recall scoring
  * Min Max Scaler, Select K Best, and Ada Boost using recall scoring
  
For the final grid search, I gave select k best a k range of 1 through 15, as I knew I wanted to remove a mininum of one feature and would allow the classifier to determine the best number of features to keep depending on each feature's importance score. For the ada boost classifier, I provided the grid search with potential estimator counts of 15 through 70 with a 5 point difference between each.  The ada boost classifier also was provided learning rate options of a range from 0.1 to 1 with a .1 difference between each.

#### Validation
For machine learning, validation is as simple as running the fitted algorithm against data that was not used to fit the algorithm, then calculating the confusion matrix and derivations from the confusion matrix to determine how successful the algorithm is on data it hasn't seen before.  A classic mistake with validation is to test and train on the same data set.  If you've written your code knowing all the pitfalls and issues in the data, then you won't be able to truly know how the code will perform when it sees 'new' data that it wasn't specifically written or tuned for.

I initially validated the simple SVC tests using a train/test split, where I fit on training data then tested on test data.  Because there are so few persons of interest in the data set, I felt that a simple train/test split would be too likely to generate groups with low or no persons of interest.  Therefore, I decided to use a Stratified Shuffle Split on the pipeline testing to ensure that each individual group had a reasonable number of persons of interest and to generate a bit of variance for the grid search.  I then ran test_classifier from python's tester package on the grid search's best estimator.

#### Evaluation
Because I used the classifier provided in python's tester package, I had a great range of statistical measures determined from the confusion matrix for each of my tested pipelines.

The first measure returned from the test classifier, accuracy, is essentially the percentage of values that are correctly classified of the tested set.  For the final classifier I selected, the accuracy was 0.85927.  This means that approximately 85.93% of the data set was classified accurately.

Because I prefer to err on the side of caution, and "trust but verify" that persons could be committing fradulent activity, I would rather have more false positives and less false negatives.  Because of this, I was preferring the recall score, otherwise known as the probability of detection.  Recall is essentially "how complete the results are" (https://en.wikipedia.org/wiki/Precision_and_recall).  Specifically, recall measures the number of true positives divided by the total positives in the data set.  In this specific set, that would be the number of persons we correctly assumed were persons of interest divided by the total number of persons of interest.  For the final classifier, the recall was 0.33750; 33.75% of the persons of interest were correctly identified using the final classifier.

#### Reference List
For transparency I'm including everything I referenced, or read, while working on this project; regardless of whether it directly affected code in the project.

"1.11. Ensemble Methods — Scikit-Learn 0.18.1 Documentation". Scikit-learn.org. N.p., 2017. Web. 9 May 2017.

"4.1. Pipeline And Featureunion: Combining Estimators — Scikit-Learn 0.18.1 Documentation". Scikit-learn.org. N.p., 2017. Web. 13 May 2017.

"Adaboostclassifier With Different Base Learners". Stackoverflow.com. N.p., 2017. Web. 9 May 2017.

"Automatically Play Sound In Ipython Notebook". Stackoverflow.com. N.p., 2017. Web. 9 May 2017.

Grus, Joel. Data Science From Scratch. Sebastopol, Calif.: O'Reilly, 2015. Print.

"How To Check If All Values Of A Dictionary Are 0, In Python?". Stackoverflow.com. N.p., 2017. Web. 13 May 2017.

"How To Ignore Deprecation Warnings In Python". Stackoverflow.com. N.p., 2017. Web. 9 May 2017.

"How To Use A Decimal Range() Step Value?". Stackoverflow.com. N.p., 2017. Web. 9 May 2017.

P, Adam. "Markdown Cheatsheet". GitHub. N.p., 2017. Web. 9 May 2017.

Scikit-Learn Algorithm Cheat-Sheet. 2017. Web. 9 May 2017.

"Sklearn.Ensemble.Adaboostclassifier — Scikit-Learn 0.18.1 Documentation". Scikit-learn.org. N.p., 2017. Web. 9 May 2017.

"Sklearn.Model_Selection.Gridsearchcv — Scikit-Learn 0.18.1 Documentation". Scikit-learn.org. N.p., 2017. Web. 9 May 2017.

"Sklearn.Model_Selection.Stratifiedshufflesplit — Scikit-Learn 0.18.1 Documentation". Scikit-learn.org. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.

"Udacity | Free Online Courses & Nanodegree Programs - Udacity". Discussions.udacity.com. N.p., 2017. Web. 9 May 2017.